In [7]:
import io
import numpy as np
import pandas as pd
from google.colab import files
import matplotlib.pyplot as plt


In [2]:
uploaded = files.upload()


Saving dataset_phishing.csv to dataset_phishing.csv


In [3]:
dat = pd.read_csv(io.BytesIO(uploaded['dataset_phishing.csv']))
dat.head()

,url,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,37,19,0,3,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,77,23,1,1,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,https://support-appleld.com.secureupdate.duila...,126,50,1,4,1,0,1,2,0,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,http://rgipt.ac.in,18,11,0,2,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,55,15,0,2,2,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,legitimate


In [5]:
dat.shape

(11430, 89)

In [6]:
dat['target'] = pd.get_dummies(dat['status'])['legitimate'].astype('int')
dat.drop('status',axis = 1, inplace=True)
dat[['url','target']].head(5)

,url,target
0,http://www.crestonwood.com/router.php,1
1,http://shadetreetechnology.com/V4/validation/a...,0
2,https://support-appleld.com.secureupdate.duila...,0
3,http://rgipt.ac.in,1
4,http://www.iracing.com/tracks/gateway-motorspo...,1


In [23]:
X = dat.iloc[: , 1:-1]

y = dat['target']
X.head()


,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,nb_eq,...,empty_title,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank
0,37,19,0,3,0,0,0,0,0,0,...,0,0,1,0,45,-1,0,1,1,4
1,77,23,1,1,0,0,0,0,0,0,...,0,1,0,0,77,5767,0,0,1,2
2,126,50,1,4,1,0,1,2,0,3,...,0,1,0,0,14,4004,5828815,0,1,0
3,18,11,0,2,0,0,0,0,0,0,...,0,1,0,0,62,-1,107721,0,0,3
4,55,15,0,2,2,0,0,0,0,0,...,0,0,1,0,224,8175,8725,0,0,6


In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3)
print(X_train.shape)
print(y_test.shape)

(8001, 87)
(3429,)


In [25]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
train_x = scaler.fit_transform(X_train.values)
test_x =  scaler.fit_transform(X_test.values)

In [114]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(max_iter=300, activation='relu',hidden_layer_sizes=200)
mlp

MLPClassifier(hidden_layer_sizes=200, max_iter=300)

In [115]:
mlp.fit(X_train,y_train)
pred = mlp.predict(X_test)

In [116]:
from sklearn.metrics import classification_report,confusion_matrix

confusion_matrix(y_test,pred)

array([[1555,  168],
       [ 335, 1371]])

In [117]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.82      0.90      0.86      1723
           1       0.89      0.80      0.84      1706

    accuracy                           0.85      3429
   macro avg       0.86      0.85      0.85      3429
weighted avg       0.86      0.85      0.85      3429

